<a href="https://colab.research.google.com/github/AlvaroDelgadillo/deep-reinforcement-learning_twin_delayed_DDPG_algorithm/blob/master/TD3_HalfCheetah.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Reinforcement Learning: Twin Delayed DDPG Algorithm

## Instalando los paquetes

In [1]:
!pip install pybullet

     |████████████████████████████████| 100.8MB 90kB/s 


## Importando las bibliotecas

In [2]:
import os
import time
import random
import numpy as np
import matplotlib.pyplot as plt
import pybullet_envs
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
from gym import wrappers
from torch.autograd import Variable
from collections import deque

## Paso 1: Inicializamos la experiencia la memoria de repetición

In [3]:
class ReplayBuffer(object):

  def __init__(self, max_size=1e6):
    self.storage = []
    self.max_size = max_size
    self.ptr = 0

  def add(self, transition):
    if len(self.storage) == self.max_size:
      self.storage[int(self.ptr)] = transition
      self.ptr = (self.ptr + 1) % self.max_size
    else:
      self.storage.append(transition)

  def sample(self, batch_size):
    ind = np.random.randint(0, len(self.storage), size=batch_size)
    batch_states, batch_next_states, batch_actions, batch_rewards, batch_dones = [], [], [], [], []
    for i in ind: 
      state, next_state, action, reward, done = self.storage[i]
      batch_states.append(np.array(state, copy=False))
      batch_next_states.append(np.array(next_state, copy=False))
      batch_actions.append(np.array(action, copy=False))
      batch_rewards.append(np.array(reward, copy=False))
      batch_dones.append(np.array(done, copy=False))
    return np.array(batch_states), np.array(batch_next_states), np.array(batch_actions), np.array(batch_rewards).reshape(-1, 1), np.array(batch_dones).reshape(-1, 1)

## Paso 2: Construimos una red neuronal para el modelo Actor y una red neuronal para el actor objetivo.

In [4]:
class Actor(nn.Module):
  
  def __init__(self, state_dim, action_dim, max_action):
    super(Actor, self).__init__()
    self.layer_1 = nn.Linear(state_dim, 400)
    self.layer_2 = nn.Linear(400, 300)
    self.layer_3 = nn.Linear(300, action_dim)
    self.max_action = max_action

  def forward(self, x):
    x = F.relu(self.layer_1(x))
    x = F.relu(self.layer_2(x))
    x = self.max_action * torch.tanh(self.layer_3(x))
    return x

## Paso 3: Construimos dos redes neuronales para los dos modelos críticos y dos redes neuronales para los dos objetivos críticos

In [5]:
class Critic(nn.Module):
  
  def __init__(self, state_dim, action_dim):
    super(Critic, self).__init__()
    # Definición de la primera red neuronal crítica
    self.layer_1 = nn.Linear(state_dim + action_dim, 400)
    self.layer_2 = nn.Linear(400, 300)
    self.layer_3 = nn.Linear(300, 1)
    # Definición de la segunda red neuronal crítica
    self.layer_4 = nn.Linear(state_dim + action_dim, 400)
    self.layer_5 = nn.Linear(400, 300)
    self.layer_6 = nn.Linear(300, 1)

  def forward(self, x, u):
    xu = torch.cat([x, u], 1)
    # Propagación hacia adelante en la primera red neuronal crítica
    x1 = F.relu(self.layer_1(xu))
    x1 = F.relu(self.layer_2(x1))
    x1 = self.layer_3(x1)
    
    # Propagación hacia adelante en la segunda red neuronal crítica    
    x2 = F.relu(self.layer_4(xu))
    x2 = F.relu(self.layer_5(x2))
    x2 = self.layer_6(x2)
    return x1, x2

  def Q1(self, x, u):
    xu = torch.cat([x, u], 1)
    x1 = F.relu(self.layer_1(xu))
    x1 = F.relu(self.layer_2(x1))
    x1 = self.layer_3(x1)
    return x1

## Pasos 4 a 15: Proceso de entrenamiento

In [6]:
# Seleccionar el dispositivo (CPU o GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Integrar todo el proceso de entrenamiento en una clase

class TD3(object):
  
  def __init__(self, state_dim, action_dim, max_action):
    self.actor = Actor(state_dim, action_dim, max_action).to(device)
    self.actor_target = Actor(state_dim, action_dim, max_action).to(device)
    self.actor_target.load_state_dict(self.actor.state_dict())
    self.actor_optimizer = torch.optim.Adam(self.actor.parameters())
    self.critic = Critic(state_dim, action_dim).to(device)
    self.critic_target = Critic(state_dim, action_dim).to(device)
    self.critic_target.load_state_dict(self.critic.state_dict())
    self.critic_optimizer = torch.optim.Adam(self.critic.parameters())
    self.max_action = max_action

  def select_action(self, state):
    state = torch.Tensor(state.reshape(1, -1)).to(device)
    return self.actor(state).cpu().data.numpy().flatten()

  def train(self, replay_buffer, iterations, batch_size=100, discount=0.99, tau=0.005, policy_noise=0.2, noise_clip=0.5, policy_freq=2):
    
    for it in range(iterations):
      
      # Paso 4: tomamos muestras de un lote de transiciones (s, s ', a, r) de la memoria.
      batch_states, batch_next_states, batch_actions, batch_rewards, batch_dones = replay_buffer.sample(batch_size)
      state = torch.Tensor(batch_states).to(device)
      next_state = torch.Tensor(batch_next_states).to(device)
      action = torch.Tensor(batch_actions).to(device)
      reward = torch.Tensor(batch_rewards).to(device)
      done = torch.Tensor(batch_dones).to(device)
      
      # Paso 5: desde el siguiente estado s’, el actor objetivo juega la siguiente acción a’
      next_action = self.actor_target(next_state)
      
      # Paso 6: Agregamos ruido gaussiano a la siguiente acción a ’y lo ajustamos en un rango de valores admitidos por el entorno.
      noise = torch.Tensor(batch_actions).data.normal_(0, policy_noise).to(device)
      noise = noise.clamp(-noise_clip, noise_clip)
      next_action = (next_action + noise).clamp(-self.max_action, self.max_action)
      
      # Paso 7: Los dos objetivos críticos toman cada uno de los pares (s’, a’) como entrada y devuelven dos valores Q, Qt1 (s’, a’) y Qt2 (s’, a’) como salidas
      target_Q1, target_Q2 = self.critic_target(next_state, next_action)
      
      # Paso 8: Mantenemos el mínimo de estos dos valores Q:min(Qt1, Qt2)
      target_Q = torch.min(target_Q1, target_Q2)
      
      # Paso 9: Obtenemos el objetivo final de los dos modelos críticos, que es: Qt=r+γ*min(Qt1, Qt2), donde γ es el factor de descuento
      target_Q = reward + ((1 - done) * discount * target_Q).detach()
      
      # Paso 10: Los dos modelos criticos toman cada uno de los pares (s, a) como entrada y devuelven dos valores Q: Q1(s,a) y Q2 (s,a) como salidas
      current_Q1, current_Q2 = self.critic(state, action)
      
      # Paso 11: Calculamos la pérdida proveniente de los dos modelos criticos: Critic Loss = MSE_Loss(Q1(s,a), Qt) + MSE_Loss(Q2(s,a), Qt)
      critic_loss = F.mse_loss(current_Q1, target_Q) + F.mse_loss(current_Q2, target_Q)
      
      # Paso 12: Retropropagamos esta pérdida crítica y actualizamos los parámetros de los dos modelos criticos con un optimizador SGD
      self.critic_optimizer.zero_grad()
      critic_loss.backward()
      self.critic_optimizer.step()
      
      # Paso 13: Una vez cada dos iteraciones, actualizamos nuestro modelo Actor realizando un ascenso de gradiente en la salida del primer modelo critico
      if it % policy_freq == 0:
        actor_loss = -self.critic.Q1(state, self.actor(state)).mean()
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()
        
        # Paso 14: Aún una vez cada dos iteraciones, actualizamos los pesos del actor objetivo mediante el promedio polyak
        for param, target_param in zip(self.actor.parameters(), self.actor_target.parameters()):
          target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)
        
        # Paso 15: Aún una vez cada dos iteraciones, actualizamos los pesos del objetivo crítico mediante el promedio de polyak
        for param, target_param in zip(self.critic.parameters(), self.critic_target.parameters()):
          target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)
  
  # Hacer un método de guardado para guardar un modelo entrenado
  def save(self, filename, directory):
    torch.save(self.actor.state_dict(), '%s/%s_actor.pth' % (directory, filename))
    torch.save(self.critic.state_dict(), '%s/%s_critic.pth' % (directory, filename))
  
  # Hacer un método de carga para cargar un modelo previamente entrenado
  def load(self, filename, directory):
    self.actor.load_state_dict(torch.load('%s/%s_actor.pth' % (directory, filename)))
    self.critic.load_state_dict(torch.load('%s/%s_critic.pth' % (directory, filename)))

## Hacemos una función que evalúa la póliza calculando su recompensa promedio en 10 episodios

In [7]:
def evaluate_policy(policy, eval_episodes=10):
  avg_reward = 0.
  for _ in range(eval_episodes):
    obs = env.reset()
    done = False
    while not done:
      action = policy.select_action(np.array(obs))
      obs, reward, done, _ = env.step(action)
      avg_reward += reward
  avg_reward /= eval_episodes
  print ("---------------------------------------")
  print ("Recompensa promedio sobre el paso de evaluación: %f" % (avg_reward))
  print ("---------------------------------------")
  return avg_reward

## Establecemos los parámetros

In [16]:
env_name = "HalfCheetahBulletEnv-v0" # Nombre de un entorno (configúrelo en cualquier entorno continuo que desee)
seed = 0 # Semilla aleatorio
start_timesteps = 1e4 # Número de iteraciones / pasos de tiempo antes de los cuales el modelo elige aleatoriamente una acción y después de los cuales comienza a usar la red de políticas
eval_freq = 5e3 # Con qué frecuencia se realiza el paso de evaluación (después de cuántos pasos de tiempo)
max_timesteps = 1e1 # Número total de iteraciones / intervalos de tiempo, se recomienda una numero superior o igual a 5e5
save_models = True # Comprobador booleano para guardar o no el modelo preentrenado
expl_noise = 0.1 # Ruido de exploración: valor STD del ruido gaussiano de exploración
batch_size = 100 # Tamaño del lote
discount = 0.99 # Factor de descuento gamma, utilizado en el cálculo de la recompensa total con descuento
tau = 0.005 # Tasa de actualización de la red objetivo
policy_noise = 0.2 # STD de ruido gaussiano agregado a las acciones con fines de exploración
noise_clip = 0.5 # Valor máximo del ruido gaussiano agregado a las acciones (política)
policy_freq = 2 # Número de iteraciones a esperar antes de que se actualice la red de políticas (modelo Actor)

## Creamos un nombre de archivo para los dos modelos guardados: los modelos Actor y Critic

In [17]:
file_name = "%s_%s_%s" % ("TD3", env_name, str(seed))
print ("---------------------------------------")
print ("Configuraciones: %s" % (file_name))
print ("---------------------------------------")

---------------------------------------
Configuraciones: TD3_HalfCheetahBulletEnv-v0_0
---------------------------------------


## Creamos una carpeta dentro de la cual se guardarán los modelos entrenados

In [18]:
if not os.path.exists("./results"):
  os.makedirs("./results")
if save_models and not os.path.exists("./pytorch_models"):
  os.makedirs("./pytorch_models")

## Creamos el entorno PyBullet

In [19]:
env = gym.make(env_name)

/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


## Sembramos semillas y obtenemos la información necesaria sobre los estados y acciones en el entorno elegido

In [20]:
env.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
max_action = float(env.action_space.high[0])

## Creamos la red de políticas (el modelo Actor)

In [21]:
policy = TD3(state_dim, action_dim, max_action)

## Creamos la memoria de repetición experimental

In [22]:
replay_buffer = ReplayBuffer()

## Definimos una lista donde se almacenan todos los resultados de la evaluación de más de 10 episodios

In [23]:
evaluations = [evaluate_policy(policy)]

---------------------------------------
Recompensa promedio sobre el paso de evaluación: -1429.426163
---------------------------------------


## Creamos un nuevo directorio de carpetas en el que se poblarán los resultados finales (videos del agente)

In [24]:
def mkdir(base, name):
    path = os.path.join(base, name)
    if not os.path.exists(path):
        os.makedirs(path)
    return path
work_dir = mkdir('exp', 'brs')
monitor_dir = mkdir(work_dir, 'monitor')
max_episode_steps = env._max_episode_steps
save_env_vid = False
if save_env_vid:
  env = wrappers.Monitor(env, monitor_dir, force = True)
  env.reset()

## Inicializamos las variables

In [25]:
total_timesteps = 0
timesteps_since_eval = 0
episode_num = 0
done = True
t0 = time.time()

## Entrenamiento

In [27]:
# Iniciamos el ciclo principal con más de max_timesteps (500,000) pasos de tiempo
while total_timesteps < max_timesteps:
  
  # Si el episodio está terminado
  if done:

    # Si no estamos al principio, comenzamos el proceso de entrenamiento del modelo.
    if total_timesteps != 0:
      print("Total Pasos de tiempo: {} Número de episodio: {} Recompensa: {}".format(total_timesteps, episode_num, episode_reward))
      policy.train(replay_buffer, episode_timesteps, batch_size, discount, tau, policy_noise, noise_clip, policy_freq)

    # Evaluamos el episodio y guardamos la política
    if timesteps_since_eval >= eval_freq:
      timesteps_since_eval %= eval_freq
      evaluations.append(evaluate_policy(policy))
      policy.save(file_name, directory="./pytorch_models")
      np.save("./results/%s" % (file_name), evaluations)
    
    # Cuando finaliza los pasos de entrenamiento, restablecemos el estado del entorno
    obs = env.reset()
    
    # Establecer el hecho a falso
    done = False
    
    # Establecer recompensas y tiempos de episodios a cero
    episode_reward = 0
    episode_timesteps = 0
    episode_num += 1
  
  # Antes de 10000 pasos de tiempo, jugamos acciones aleatorias
  if total_timesteps < start_timesteps:
    action = env.action_space.sample()
  else: # Después de 10000 pasos de tiempo, cambiamos al modelo
    action = policy.select_action(np.array(obs))
    # Si el parámetro explore_noise no es 0, agregamos ruido a la acción y lo recortamos
    if expl_noise != 0:
      action = (action + np.random.normal(0, expl_noise, size=env.action_space.shape[0])).clip(env.action_space.low, env.action_space.high)
  
  # El agente realiza la acción en el entorno, luego alcanza el siguiente estado y recibe la recompensa
  new_obs, reward, done, _ = env.step(action)
  
  # Comprobamos si el episodio está terminado
  done_bool = 0 if episode_timesteps + 1 == env._max_episode_steps else float(done)
  
  # Aumentamos la recompensa total
  episode_reward += reward
  
  # Almacenamos la nueva transición en la memoria de experiencia de repetición (ReplayBuffer)
  replay_buffer.add((obs, new_obs, action, reward, done_bool))

  # Actualizamos el estado, el intervalo de tiempo del episodio, el intervalo de tiempo total y los intervalos de tiempo desde la evaluación de la política.
  obs = new_obs
  episode_timesteps += 1
  total_timesteps += 1
  timesteps_since_eval += 1

# Agregamos la última evaluación de políticas a nuestra lista de evaluaciones y guardamos nuestro modelo
evaluations.append(evaluate_policy(policy))
if save_models: policy.save("%s" % (file_name), directory="./pytorch_models")
np.save("./results/%s" % (file_name), evaluations)

---------------------------------------
Recompensa promedio sobre el paso de evaluación: -1435.223787
---------------------------------------


## Inferencia

In [29]:
class Actor(nn.Module):
  
  def __init__(self, state_dim, action_dim, max_action):
    super(Actor, self).__init__()
    self.layer_1 = nn.Linear(state_dim, 400)
    self.layer_2 = nn.Linear(400, 300)
    self.layer_3 = nn.Linear(300, action_dim)
    self.max_action = max_action

  def forward(self, x):
    x = F.relu(self.layer_1(x))
    x = F.relu(self.layer_2(x))
    x = self.max_action * torch.tanh(self.layer_3(x)) 
    return x

class Critic(nn.Module):
  
  def __init__(self, state_dim, action_dim):
    super(Critic, self).__init__()
    # Definición de la primera red neuronal crítica
    self.layer_1 = nn.Linear(state_dim + action_dim, 400)
    self.layer_2 = nn.Linear(400, 300)
    self.layer_3 = nn.Linear(300, 1)
    # Definición de la segunda red neuronal crítica
    self.layer_4 = nn.Linear(state_dim + action_dim, 400)
    self.layer_5 = nn.Linear(400, 300)
    self.layer_6 = nn.Linear(300, 1)

  def forward(self, x, u):
    xu = torch.cat([x, u], 1)
    # Propagación hacia adelante en la primera red neuronal crítica
    x1 = F.relu(self.layer_1(xu))
    x1 = F.relu(self.layer_2(x1))
    x1 = self.layer_3(x1)
    # Propagación hacia adelante en la segunda red neuronal crítica
    x2 = F.relu(self.layer_4(xu))
    x2 = F.relu(self.layer_5(x2))
    x2 = self.layer_6(x2)
    return x1, x2

  def Q1(self, x, u):
    xu = torch.cat([x, u], 1)
    x1 = F.relu(self.layer_1(xu))
    x1 = F.relu(self.layer_2(x1))
    x1 = self.layer_3(x1)
    return x1

# Seleccionar el dispositivo (CPU o GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Integrar todo el proceso de formación en una clase

class TD3(object):
  
  def __init__(self, state_dim, action_dim, max_action):
    self.actor = Actor(state_dim, action_dim, max_action).to(device)
    self.actor_target = Actor(state_dim, action_dim, max_action).to(device)
    self.actor_target.load_state_dict(self.actor.state_dict())
    self.actor_optimizer = torch.optim.Adam(self.actor.parameters())
    self.critic = Critic(state_dim, action_dim).to(device)
    self.critic_target = Critic(state_dim, action_dim).to(device)
    self.critic_target.load_state_dict(self.critic.state_dict())
    self.critic_optimizer = torch.optim.Adam(self.critic.parameters())
    self.max_action = max_action

  def select_action(self, state):
    state = torch.Tensor(state.reshape(1, -1)).to(device)
    return self.actor(state).cpu().data.numpy().flatten()

  def train(self, replay_buffer, iterations, batch_size=100, discount=0.99, tau=0.005, policy_noise=0.2, noise_clip=0.5, policy_freq=2):
    
    for it in range(iterations):
      
      # Paso 4: tomamos muestras de un lote de transiciones (s, s', a, r) de la memoria
      batch_states, batch_next_states, batch_actions, batch_rewards, batch_dones = replay_buffer.sample(batch_size)
      state = torch.Tensor(batch_states).to(device)
      next_state = torch.Tensor(batch_next_states).to(device)
      action = torch.Tensor(batch_actions).to(device)
      reward = torch.Tensor(batch_rewards).to(device)
      done = torch.Tensor(batch_dones).to(device)
      
      # Paso 5: desde el siguiente estado s’, el actor objetivo juega la siguiente acción a’
      next_action = self.actor_target(next_state)
      
      # Paso 6: Agregamos ruido gaussiano a la siguiente acción a’ y lo ajustamos en un rango de valores admitidos por el entorno.
      noise = torch.Tensor(batch_actions).data.normal_(0, policy_noise).to(device)
      noise = noise.clamp(-noise_clip, noise_clip)
      next_action = (next_action + noise).clamp(-self.max_action, self.max_action)
      
      # Paso 7: Los dos objetivos críticos toman cada uno de los pares (s’, a’) como entrada y devuelven dos valores Q: Qt1(s’, a’) y Qt2 (s’, a’) como salidas
      target_Q1, target_Q2 = self.critic_target(next_state, next_action)
      
      # Paso 8: Mantenemos el mínimo de estos dos valores Q: min(Qt1, Qt2)
      target_Q = torch.min(target_Q1, target_Q2)
      
      # Paso 9: Obtenemos el objetivo final de los dos modelos críticos, que es: Qt=r+γ*min(Qt1, Qt2), donde γ es el factor de descuento
      target_Q = reward + ((1 - done) * discount * target_Q).detach()
      
      # Paso 10: Los dos modelos criticos toman cada uno de los pares (s, a) como entrada y devuelven dos valores Q: Q1(s, a) y Q2(s, a) como salidas
      current_Q1, current_Q2 = self.critic(state, action)
      
      # Paso 11: Calculamos la pérdida proveniente de los dos modelos Criticos: Critic Loss= MSE_Loss(Q1(s, a), Qt) + MSE_Loss(Q2(s, a), Qt)
      critic_loss = F.mse_loss(current_Q1, target_Q) + F.mse_loss(current_Q2, target_Q)
      
      # Paso 12: Retropropagamos esta pérdida crítica y actualizamos los parámetros de los dos modelos Critic con un optimizador SGD
      self.critic_optimizer.zero_grad()
      critic_loss.backward()
      self.critic_optimizer.step()
      
      # Paso 13: Una vez cada dos iteraciones, actualizamos nuestro modelo Actor realizando un ascenso de gradiente en la salida del primer modelo Critico
      if it % policy_freq == 0:
        actor_loss = -self.critic.Q1(state, self.actor(state)).mean()
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()
        
        # Paso 14: Aún una vez cada dos iteraciones, actualizamos los pesos del actor objetivo mediante el promedio polyak
        for param, target_param in zip(self.critic.parameters(), self.critic_target.parameters()):
          target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)
        
        # Paso 15: Aún una vez cada dos iteraciones, actualizamos los pesos del objetivo crítico mediante el promedio polyak
        for param, target_param in zip(self.actor.parameters(), self.actor_target.parameters()):
          target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)
  
  # Hacer un método de guardado para guardar un modelo entrenado
  def save(self, filename, directory):
    torch.save(self.actor.state_dict(), '%s/%s_actor.pth' % (directory, filename))
    torch.save(self.critic.state_dict(), '%s/%s_critic.pth' % (directory, filename))
  
  # Hacer un método de carga para cargar un modelo previamente entrenado
  def load(self, filename, directory):
    self.actor.load_state_dict(torch.load('%s/%s_actor.pth' % (directory, filename)))
    self.critic.load_state_dict(torch.load('%s/%s_critic.pth' % (directory, filename)))

def evaluate_policy(policy, eval_episodes=10):
  avg_reward = 0.
  for _ in range(eval_episodes):
    obs = env.reset()
    done = False
    while not done:
      action = policy.select_action(np.array(obs))
      obs, reward, done, _ = env.step(action)
      avg_reward += reward
  avg_reward /= eval_episodes
  print ("---------------------------------------")
  print ("Recompensa promedio sobre el paso de evaluación: %f" % (avg_reward))
  print ("---------------------------------------")
  return avg_reward

env_name = "HalfCheetahBulletEnv-v0"
seed = 0

file_name = "%s_%s_%s" % ("TD3", env_name, str(seed))
print ("---------------------------------------")
print ("Configuraciones: %s" % (file_name))
print ("---------------------------------------")

eval_episodes = 10
save_env_vid = True
env = gym.make(env_name)
max_episode_steps = env._max_episode_steps
if save_env_vid:
  env = wrappers.Monitor(env, monitor_dir, force = True)
  env.reset()
env.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
max_action = float(env.action_space.high[0])
policy = TD3(state_dim, action_dim, max_action)
policy.load(file_name, './pytorch_models/')
_ = evaluate_policy(policy, eval_episodes=eval_episodes)

---------------------------------------
Configuraciones: TD3_HalfCheetahBulletEnv-v0_0
---------------------------------------


/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


---------------------------------------
Recompensa promedio sobre el paso de evaluación: -1429.427324
---------------------------------------
